<a href="https://colab.research.google.com/github/Parzival-Prime/Fish-vs-Cat-Classifier/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [ ]:
# !kaggle datasets download zuraiz/fish-vs-cats-imagenet-subdataset

Dataset URL: https://www.kaggle.com/datasets/zuraiz/fish-vs-cats-imagenet-subdataset
License(s): unknown
  0% 0.00/67.7M [00:00<?, ?B/s]
100% 67.7M/67.7M [00:00<00:00, 868MB/s]


In [ ]:
# import zipfile
# zipref = zipfile.ZipFile('fish-vs-cats-imagenet-subdataset.zip', 'r')
# zipref.extractall()
# zipref.close()

### Prepare Dataset

In [ ]:
import torch
import torchvision
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
train_data_path = os.path.join('Fish-vs-Cats', 'train')
test_data_path = os.path.join('Fish-vs-Cats', 'test')
val_data_path = os.path.join('Fish-vs-Cats', 'val')

In [ ]:
transforms = v2.Compose([
    v2.Resize((90, 90)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
from PIL import Image, UnidentifiedImageError
class SafeImageFolder(torchvision.datasets.ImageFolder):
  def __getitem__(self, index):
    path, target = self.samples[index]
    try:
      sample = self.loader(path)
    except UnidentifiedImageError:
      return self.__getitem__((index + 1) % len(self.samples))
    if self.transfom is not None:
      sample = self.transform(sample)
    return sample, target

In [ ]:
train_data = SafeImageFolder(train_data_path, transforms)
test_data = SafeImageFolder(test_data_path, transforms)
val_data = SafeImageFolder(val_data_path, transforms)

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

## Creating Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cpu


In [ ]:
transfer_model = models.ResNet50(pretrained=True)

In [ ]:
for name, param in transfer_model.named_parameters():
  if "bn" not in name:
    param.requires_grad = False

In [ ]:
transfer_model.fc = nn.Sequential(
    nn.Linear(transfer_model.fc.in_features, 500),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(500, 2)
)

In [ ]:
model = CNNNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 20

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
  for epoch in range(epochs):
    model.train()
    training_loss = 0
    val_loss = 0
    train_correct = 0
    train_total = 0
    val_correct = 0
    val_total = 0

    for X_train, y_train in train_loader:
      X_train, y_train = X_train.to(device), y_train.to(device)
      optimizer.zero_grad()
      y_pred = model(X_train)
      loss = criterion(y_pred, y_train)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item()
      correct = torch.eq(torch.max(F.Softmax(y_pred, dim=1), dim=1)[1], y_train).view(-1)
      train_correct += torch.sum(correct).item()
      train_total += correct.shape[0]
    training_loss /= len(train_loader)

    model.eval()
    for X_val, y_val in val_loader:
      X_val, y_val = X_val.to(device), y_val.to(device)
      optimizer.zero_grad()
      y_pred = model(X_val)
      loss = criterion(y_pred, y_val)
      loss.backward()
      optimizer.step()
      val_loss = loss.data.item()
      correct = torch.eq(torch.max(F.Softmax(y_pred, dim=1), dim=1)[1], y_val).view(-1)
      val_correct += torch.sum(correct).item()
      val_total += correct.shape[0]
    val_loss /= len(val_loader)

    print(f'Epoch: {epoch}, Training Loss: {training_loss:.2f}, Training_accuracy: {((train_correct / train_total) * 100):.2f},  Validation Loss: {val_loss:.2f}, val_accuracy = {((val_correct / val_total) * 100):.2f}%')


In [ ]:
torch.save(simplenet, "model/Fine-tuned-ResNet50.pkl")